In [1]:
from glob import glob
import pandas as pd
import numpy as np
import re
import cv2
from tqdm import tqdm

In [13]:
outputs = glob('outputs/*')
# frames = glob('cam1/*')
frame_nums = []

for i in outputs:
    frame_nums.append(re.findall("\d+", i)[0])
    
table = pd.DataFrame(frame_nums, columns=['frame_num'])
table['output'] = pd.Series(outputs)
table['raw'] = 'cam1/' + table['frame_num'] + '.jpg'
table['frame_num'] = table['frame_num'].astype(int)
table['homo'] = table['raw'].str.replace('cam1', 'images')
table['label'] = table['raw'].str.replace('.jpg', '.txt').str.replace('cam1', 'labels')

table.sort_values('frame_num').reset_index(drop=True)

table.sort_values('frame_num', inplace=True)
table.reset_index(drop=True, inplace=True)

pd.to_pickle(table, 'table.pkl')

In [16]:
pts_src1 = np.array([[614, 294], [581, 326],[503, 407], [313, 348], [478, 345]])
pts_dst = np.array([[650, 550], [600, 550], [500, 550],[500, 400], [550, 500]]) #these are the zoomed out destination points
homo = cv2.findHomography(pts_src1, pts_dst)[0]

y = np.empty((len(table), 17, 2))

for i, row in tqdm(table.iterrows(), total=len(table)):
    undis = cv2.imread(row.raw)
    transformed = cv2.warpPerspective(undis, homo, (undis.shape[1], undis.shape[0]))
    cv2.imwrite(row.homo, transformed)
    
    points = pd.read_csv(table.iloc[0].output).values[:, 1:]
    points = cv2.perspectiveTransform(points.reshape(-1, 1, 2).astype(np.float32), homo)
    points /= transformed.shape[:2] #scale to the dimensions of the image
    y[i] = points.reshape(17, 2)
    
    with open(row.label, 'w') as f:
        for xcoord, ycoord in y[i]:
            f.write(f'0 {xcoord} {ycoord} 0 0\n')
    
np.save('y.npy', y)

100%|██████████| 1633/1633 [00:35<00:00, 45.43it/s]
